In [ ]:
def calculate_metrics(y_true, y_pred):
    # Check if y_true is a pandas DataFrame and convert it to a NumPy array for compatibility
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.to_numpy()  # Convert y_true to a NumPy array

    # Check if y_pred is a pandas DataFrame and convert it to a NumPy array for compatibility
    if isinstance(y_pred, pd.DataFrame):
        y_pred = y_pred.to_numpy()  # Convert y_pred to a NumPy array

    # Check if y_true is already flattened (1D)
    if y_true.ndim > 1:
        y_true = y_true.flatten()  # Flatten if it has more than 1 dimension

    # Check if y_pred is already flattened (1D)
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()  # Flatten if it has more than 1 dimension

    # Find unique class names in y_true and determine the number of unique classes
    class_names = np.unique(y_true)  # Get unique class names from y_true
    unique_classes = class_names.size  # Count the number of unique classes

    # Initialize a confusion matrix with zeros, sized based on the number of unique classes
    confusion_matrix = np.zeros((unique_classes, unique_classes), dtype=int)

    # Map class names to indices for easy lookup
    class_name_to_index = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Count occurrences of actual vs predicted labels
    for actual, predicted in zip(y_true, y_pred):
        # Ensure actual and predicted are scalar values, not arrays
        actual = actual.item() if isinstance(actual, np.ndarray) else actual
        predicted = predicted.item() if isinstance(predicted, np.ndarray) else predicted
        
        # Find the index for the actual and predicted class
        actual_index = class_name_to_index[actual]
        predicted_index = class_name_to_index[predicted]

        # Increment the appropriate cell in the confusion matrix
        confusion_matrix[actual_index, predicted_index] += 1

    # Print the confusion matrix row by row
    print("\nConfusion matrix:")
    for row in confusion_matrix:
        print(" ".join(map(str, row)))  # Print each row of the confusion matrix

    # --- --- --- --- --- ---
    
    # Accuracy Calculation
    
    # Sum of the diagonal elements (correct predictions)
    correct_predictions = np.trace(confusion_matrix)  # np.trace() gives the sum of diagonal elements

    # Total number of predictions (sum of all elements in the matrix)
    total_predictions = np.sum(confusion_matrix)

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions
    print(f"\nAccuracy: {accuracy}")

    # --- --- --- --- --- ---
        
    # Precision Calculation

    def calculate_precision(confusion_matrix, class_names):
        precision = {}
        
        # Iterate over each class to calculate its precision
        for i, class_name in enumerate(class_names):
            # True Positive (TP) is the value in the diagonal for that class
            true_positive = confusion_matrix[i, i]
            
            # False Positive (FP) is the sum of the column (excluding the diagonal)
            false_positive = np.sum(confusion_matrix[:, i]) - true_positive
            
            # Precision for the current class
            precision[class_name] = true_positive / (true_positive + false_positive) if (true_positive + false_positive) != 0 else 0        
        
        return precision
    
    precision = calculate_precision(confusion_matrix, class_names)
    print("\nPrecision for each class:")
    for class_name, precision_value in precision.items():
        print(f"{class_name}: {precision_value:}")

    total_precision = sum(precision.values())
    print(f"\nMacro precision: {total_precision / unique_classes}")

    # --- --- --- --- --- ---
    
    # Recall Calculation
    
    def calculate_recall(confusion_matrix, class_names):
        recall = {}
        
        # Iterate over each class to calculate its recall
        for i, class_name in enumerate(class_names):
            # True Positive (TP) is the value in the diagonal for that class
            true_positive = confusion_matrix[i, i]
            
            # False Negative (FN) is the sum of the row (excluding the diagonal)
            false_negative = np.sum(confusion_matrix[i, :]) - true_positive
            
            # Recall for the current class
            recall[class_name] = true_positive / (true_positive + false_negative) if (true_positive + false_negative) != 0 else 0        
        
        return recall
    
    recall = calculate_recall(confusion_matrix, class_names)
    print("\nRecall for each class:")
    for class_name, recall_value in recall.items():
        print(f"{class_name}: {recall_value:.4f}")

    total_recall = sum(recall.values())
    print(f"\nMacro recall: {total_recall / unique_classes}")

    # --- --- --- --- --- ---

    # F1 Score Calculation
    
    def calculate_f1_score(precision, recall):
        f1_scores = {}
    
        # Calculate F1 score for each class
        for class_name in precision.keys():
            p = precision[class_name]
            r = recall[class_name]
            
            # Calculate F1 score for the class, handling cases where p + r = 0
            f1_scores[class_name] = (2 * p * r) / (p + r) if (p + r) != 0 else 0
        
        return f1_scores
    
    f1_scores = calculate_f1_score(precision, recall)
    print("\nF1 Score for each class:")
    for class_name, f1_value in f1_scores.items():
        print(f"{class_name}: {f1_value:}")
        
    # Macro F1 Score Calculation
    total_f1 = sum(f1_scores.values())  # Sum of F1 scores for each class
    macro_f1 = total_f1 / len(f1_scores)  # Average F1 score across all classes
    
    print(f"\nMacro F1 score: {macro_f1:}")